# Highway with SB3's DQN

##  Warming up
We start with a few useful installs and imports:

In [1]:
# Install environment and agent
#!pip install highway-env
# TODO: we use the bleeding edge version because the current stable version does not support the latest gym>=0.21 versions. Revert back to stable at the next SB3 release.
#!pip install git+https://github.com/DLR-RM/stable-baselines3

# Environment
import gymnasium as gym
import highway_env

gym.register_envs(highway_env)

# Agent
from stable_baselines3 import DQN

# Visualization utils
#%load_ext tensorboard
import sys, os
from tqdm.notebook import trange
#!pip install tensorboardx gym pyvirtualdisplay
#!apt-get install -y xvfb ffmpeg
#!git clone https://github.com/Farama-Foundation/HighwayEnv.git 2> /dev/null
#sys.path.insert(0, '/content/HighwayEnv-NGSIM/scripts/')
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, parent_dir)

from scripts.utils import record_videos, show_videos

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


FileNotFoundError: [WinError 2] The system cannot find the file specified

## Training
Run tensorboard locally to visualize training.

In [ ]:
%tensorboard --logdir "highway_dqn"

In [ ]:
model = DQN('MlpPolicy', 'highway-fast-v0',
                policy_kwargs=dict(net_arch=[256, 256]),
                learning_rate=5e-4,
                buffer_size=15000,
                learning_starts=200,
                batch_size=32,
                gamma=0.8,
                train_freq=1,
                gradient_steps=1,
                target_update_interval=50,
                exploration_fraction=0.7,
                verbose=1,
                tensorboard_log='highway_dqn/')
model.learn(int(2e4))


## Testing

Visualize a few episodes

In [ ]:
env = gym.make('highway-fast-v0', render_mode='rgb_array')
env = record_videos(env)
for episode in trange(3, desc='Test episodes'):
    (obs, info), done = env.reset(), False
    while not done:
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, done, truncated, info = env.step(int(action))
env.close()
show_videos()